importing necessary libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
poly = np.polynomial.polynomial
import math
import random
import pandas as pd
import time

In [2]:
easyP=10708729

# Question 1

In [3]:
#uses repeated squaring method to calculate (a/p) using Euler's criterion
def legendre1(a,p):
    i=(p-1)//2
    answer=1
    multiplier=a%p
    while i>0:
        if i%2!=0:
            answer*=multiplier
            answer%=p
            i-=1
        i//=2
        multiplier*=multiplier
        multiplier%=p
    if answer==p-1:
        answer=-1
    return answer

#returns first the tally, then the list of 100 random numbers between 1 and p and their legendre symbols
#then returns first the tally, the the list of the legendre symbols of numbers from 1 to 100
def question1(p):
    legendreList1=[]
    answer1=0
    legendreList2=[]
    answer2=0
    for i in range(1,101):
        a=random.randint(1,p)
        l1=legendre1(a,p)
        l2=legendre1(i,p)
        if l1==1:
            answer1+=1
        legendreList1.append((a,l1))
        if l2==1:
            answer2+=1
        legendreList2.append((i,l2))
    print("The first tally of quadratic residues is "+str(answer1))
    print(legendreList1)
    print("The second tally of quadratic residues between 1 and 100 is "+str(answer2))
    print(legendreList2)

# Question 2

pretty sure this isn't the koblitz version so need to redo

In [4]:
def primeFactors(n):
    i=2
    factorList=[]
    while n>1:
        if i**2>n:
            factorList.append(n)
            return factorList
        if n%i==0:
            factorList.append(i)
            n/=i
        else:
            i+=1
    return factorList
  
def question2(a,p):
    Answer=1
    a%=p
    if a==1:
        return 1
    if a==p-1:
        if p%4==1:
            return 1
        else:
            return -1
    if a==2:
        if p%8==1 or p%8==7:
            return 1
        else:
            return -1
    factors1=primeFactors(a)
    for i in primeFactors(a):
        if i==2:
            if p%8==1 or p%8==7:
                Answer*=1
            else:
                Answer*=-1
        else:
            if p%4==3 and i%4==3:
                Answer*=-question2(p,i)
            else:
                Answer*=question2(p,i)
    return Answer
#note to self: see if there is any way of making the last line more efficient

In [5]:
#we're assuming b an odd prime, a any integer. So don't need to check if b==0, don't need to do the dividing by two.
#unfortunately that was as far as I'd read up to
def kronecker(a,b):
    k=1
    if a==0:
        return 0
    return inner(a,b,k)
     
#can assume b is odd because we made old a odd
def inner(a,b,k):
    power=0
    while a%2==0:
        a//=2
        power+=1
    if power%2!=0:
        if b%8==3 or b%8==5:
            k*=-1
    if b%4==3 and a%4==3:
        k*=-1
    if a==1:
        return k
    return inner(b%abs(a),abs(a),k)


#not fast enough, so need to take out recursive aspect


def kronecker1(a,b):
    k=1
    if a==0:
        return 0
    while b!=1:
        power=0
        while a%2==0:
            a//=2
            power+=1
        if power%2!=0:
            if b%8==3 or b%8==5:
                k*=-1
        if b%4==3:
            #even though we're testing numbers larger than 0, need this just in case
            if a<0:
                k*=-1
                a=abs(a)
            if a%4==3:
                k*=-1
        (a,b) = (b%a,a)
    return k

def kronecker2(a,b):
    a%=b
    k=1
    if a==0:
        return 0
    while b!=1:
        power=0
        while a%2==0:
            a//=2
            power+=1
        if power%2!=0:
            if b%8==3 or b%8==5:
                k*=-1
        if b%4==3:
            #even though we're testing numbers larger than 0, need this just in case
            if a<0:
                k*=-1
                a=abs(a)
            if a%4==3:
                k*=-1
        (a,b) = (b%a,a)
    return k

In [6]:
# start = time.time()
# for i in range(1,1000001):
#     kronecker(i,10708729)
# end = time.time()
# print(end - start)

start = time.time()
for i in range(10000000000000,10000000020000):
    kronecker1(i,10708729)
end = time.time()
print(end - start)

start = time.time()
for i in range(10000000000,10000020000):
    kronecker2(i,10708729)
end = time.time()
print(end - start)

# start = time.time()
# for i in range(1,100001):
#     legendre1(i,10708729)
# end = time.time()
# print(end - start)

0.1006929874420166
0.08130908012390137


# Question 4

In [7]:
def powerMod(n,power,p):
    answer=1
    iterate=n%p
    while power>0:
        if power%2!=0:
            answer*=iterate
            answer%=p
            power-=1
        power//=2
        iterate*=iterate
        iterate%=p
    return answer

# Chapter 5

    let p-1 = 2**i * c
    by Sylow, we have a 2-subgroup G of order 2**i
    by Euler's criterion, a**((p-1)/2) = (a**c)**(2**(i-1)) = 1 mod p
    (a**c) in G so say we could find a generator z of G
    then there exists k s.t. a**c * z**k = 1 mod p
    then a**(c+1) * z**(k) = a
    so set x = a**((c+1)/2) * z**(k/2)

In [8]:
#uses Fermat-Euler
def squareInverse(n,p):
    return powerMod(n,(p-3)//2,p)

def nonSquareInverse(n,p):
    return powerMod(n,p-2,p)

In [9]:
#finds alpha, s s.t. p-1 = 2**alpha * s
def decompose(p):
    alpha=0
    s=p-1
    while s%2==0:
        s//=2
        alpha+=1
    return alpha,s


def question5(a,p):
    alpha,s = decompose(p)
    n = random.randint(2,p-1)
    while kronecker1(n,p)==1:
        n = random.randint(2,p-1)
    b=powerMod(n,s,p)
    almostA = powerMod(a,s,p)
    k = almostSquare(p,b,almostA,alpha,s)
    x = powerMod(a,(s+1)//2,p)*powerMod(b,k//2,p)%p
    return x


def almostSquare(p,b,almostA,alpha,s):
    goal=squareInverse(almostA,p)
    bInverse = nonSquareInverse(b,p)
    answerList=[]
    power=1
    for i in range(1,alpha+1):
        testRun=goal
        k=0
        for j in answerList:
            if j==1:
                testRun*=powerMod(bInverse,2**k,p)%p
            k+=1
        #should change this because we're pretending p-1 = 2**alpha
        testRun=powerMod(testRun,(2**(alpha-power)),p)
        if testRun==1:
            answerList+=[0]
        else:
            answerList+=[1]
        power+=1
    answer=binary(answerList)
    return answer

def binary(answerList):
    answerList=answerList[::-1]
    answer=0
    for i in answerList:
        answer*=2
        answer+=i
    return answer

In [10]:
decompose(10708729)

(3, 1338591)

In [11]:
for i in range(1,21):
    if kronecker1(i,easyP)==1:
        j=question5(i,easyP)
        print(i,j,j**2%easyP)

1 1 1
2 1855179 2
3 9316627 3
4 2 4
5 1848833 5
6 4359214 6
8 6998371 8
9 3 9
10 4072761 10
11 9464082 11
12 2784204 12
15 3937081 15
16 10708725 16
17 7665993 17
18 5565537 18
19 8559407 19
20 7011063 20


# Question 6

In [12]:
# class Polynomials:
#     def __init__(self,coefficients):
#         self.c = coefficients
#     #returns the degree + 1
#     def length(self):
#         return len(self.c)
#     #f=f+g
#     def add(self, g):
#         a=self.c
#         b=g.c
#         if len(a)>=len(b):
#             b+=[0]*(len(a)-len(b))
#         else:
#             a+=[0]*(len(b)-len(a))
#         for i in range(0,len(a)):
#             a[i]+=b[i]
#         self.c = a
    
#     def sub(self,g):
#         a=self.c
#         b=g.c
#         if len(a)>=len(b):
#             b+=[0]*(len(a)-len(b))
#         else:
#             a+=[0]*(len(b)-len(a))
#         for i in range(0,len(a)):
#             a[i]-=b[i]
#         self.c = a
#     #f = lf
#     def scalar(self, coeff):
#         for i in range(0,len(self.c)):
#             self.c[i]*=coeff
#     #f = f.g
#     def mult(self,g):
#         answer=Polynomials([])
#         a=self.c
#         n=len(a)
#         b=g.c
#         for i in range(0,len(b)):
#             newA = list(a)
#             for j in range(0,n):
#                 newA[j]*=b[i]
#             newA = [0]*i + newA
#             newSelf=Polynomials(newA)
#             answer.add(newSelf)
#         self.c = answer.c
#     #gets rid of leading zeroes
#     def strip(self):
#         i=self.c[-1]
#         while i==0:
#             self.c = self.c[:-1:]
#             i=self.c[-1]
#     #mods coefficients by p
#     def mod(self,p):
#         a=self.c
#         for i in range(0,len(a)):
#             self.c[i]=a[i]%p
            
        

        

#we have no information about n now, so assume non square
def inverse(n,p):
    return powerMod(n,p-2,p)

In [13]:
#aim - find q, r s.t. f = qg + r
#later on decide if want to return distinct r to f or change f to r
# def divisionAlgorithm(f,g,p):
#     rList=list(f.c)
#     r=Polynomials(rList)
# #     if f.length < g.length:
# #         q=Polynomials([])
# #         r=f
# #         return q,r
#     q=Polynomials([])
#     #leading coefficient of g
#     lcg = g.c[-1]
#     lcgInverse = inverse(lcg,p)
#     while r.length() >= g.length():
#         print(r.c)
#         print(g.c)
#         fcg = r.c[-1]
#         newPoly = Polynomials([0]*(r.length()-g.length()) + [(fcg*lcgInverse)%p])
#         q.add(newPoly)
#         newPoly.mult(g)
#         r.sub(newPoly)
#         r.mod(p)
#         r.strip
#     return q,r

In [14]:
class Polynomial:
    def __init__(self,coefficients,p):
        self.c = coefficients
        self.p = p
        self.len = len(coefficients)
        if self.len>0:
            self.degree = max([power for power in coefficients])
        else: self.degree = 0
    def strip(self):
        for i in range(0,self.degree+1):
            if i in self.c and self.c[i]==0:
                del self.c[i] 
    def scalar(self,const):
        for i in self.c:
            self.c[i]*=const
    def add(self,g):
        for i in self.c:
            if i in g.c:
                self.c[i]+=g.c[i]
                self.c[i]%=self.p
        for i in g.c:
            if i not in self.c:
                self.c[i]=g.c[i]
        self.strip()
    def sub(self,g):
        for i in self.c:
            if i in g.c:
                self.c[i]-=g.c[i]
                self.c[i]%=self.p
        for i in g.c:
            if i not in self.c:
                self.c[i]=self.p-g.c[i]
        self.strip()
        self.len = len(self.c)
        if self.len>0:
            self.degree = max([power for power in self.c])
        else: self.degree = 0
    def mult(self,g):
        answer={}
        for i in self.c:
            for j in g.c:
                if i+j in answer:
                    answer[i+j]+=(self.c[i]*g.c[j])
                    answer[i+j]%=self.p
                else:
                    answer[i+j]=(self.c[i]*g.c[j])%self.p
        self.c = answer
        self.strip()

In [15]:
g = Polynomial({3:1,2:6,1:2,0:10},17)
a = Polynomial({0:1},17)
g.mult(a)
g.c

{3: 1, 2: 6, 1: 2, 0: 10}

In [16]:
def quotientIterate(f,g):
    answer=Polynomial({},f.p)
    while f.degree>=g.degree:
        print(f.c, f.degree)
        lead = f.c[f.degree]
        gLead = g.c[g.degree]
        gCoeff = (inverse(gLead,f.p)*lead)%f.p
        a = Polynomial({(f.degree-g.degree):gCoeff},f.p)
        newPoly = Polynomial(g.c,g.p)
        newPoly.mult(a)
        answer.add(newPoly)
        f.sub(newPoly)
    return answer.c, f.c

In [17]:
quotientIterate(Polynomial({3:1,2:8,1:12,0:4},17),Polynomial({3:1,2:6,1:2,0:10},17))

{3: 1, 2: 8, 1: 12, 0: 4} 3


({3: 1, 2: 6, 1: 2, 0: 10}, {2: 2, 1: 10, 0: 11})

In [18]:
def euclidPoly(f,g):
    p=f.p
    toBeDecided="notDecided"
    while g.degree>0:
        print(f.c,g.c)
        a,b = quotientIterate(Polynomial(f.c,p),Polynomial(g.c,p))
        print(f.c,g.c)
        print(a,b)
        print(f.c, "=", a, "*", g.c, "+", b)
        f, g = Polynomial(g.c,p), Polynomial(b,p)
    return

In [19]:
euclidPoly(Polynomial({3:1,2:8,1:12,0:4},17),Polynomial({3:1,2:6,1:2,0:10},17))

{3: 1, 2: 8, 1: 12, 0: 4} {3: 1, 2: 6, 1: 2, 0: 10}
{3: 1, 2: 8, 1: 12, 0: 4} 3
{2: 2, 1: 10, 0: 11} {3: 1, 2: 6, 1: 2, 0: 10}
{3: 1, 2: 6, 1: 2, 0: 10} {2: 2, 1: 10, 0: 11}
{2: 2, 1: 10, 0: 11} = {3: 1, 2: 6, 1: 2, 0: 10} * {3: 1, 2: 6, 1: 2, 0: 10} + {2: 2, 1: 10, 0: 11}
{3: 1, 2: 6, 1: 2, 0: 10} {2: 2, 1: 10, 0: 11}
{3: 1, 2: 6, 1: 2, 0: 10} 3
{2: 1, 1: 5, 0: 10} 2
{0: 13} {2: 2, 1: 10, 0: 11}
{3: 1, 2: 6, 1: 2, 0: 14} {0: 13}
{0: 13} = {3: 1, 2: 6, 1: 2, 0: 14} * {2: 2, 1: 10, 0: 11} + {0: 13}


In [20]:
# what if we give up on classes and just do it in lists?
def degree(f):
    if len(f)>0:
        return max([i for i in f])
    return -1

def strip(f,p):
    # maybe a bit inefficient to bother modding here, it should probably already be done
    for i in range(0,degree(f)+1):
        if i in f and f[i]%p==0:
            del f[i]
    return f

def dictAdd(f,g,p):
    answer = dict(f)
    for i in g:
        if i in answer:
            answer[i]+=g[i]
            answer[i]%=p
        else:
            answer[i]=g[i]
    answer = strip(answer,p)
    return answer
            
def dictScal(f,const,p):
    answer={}
    for i in f:
        answer[i]=f[i]*const%p
    return answer

def dictMult(f,g,p):
    answer={}
    for i in f:
        for j in g:
            if i+j in answer:
                answer[i+j]+=(f[i]*g[j])%p
            else:
                answer[i+j]=(f[i]*g[j])%p
    answer = strip(answer,p)
    return answer




In [21]:
def quotient(f,g,p):
    degF = degree(f)
    degG = degree(g)
    q = {}
    while degF >= degG:
        lead = f[degF]
        gLead = g[degG]
        gCoeff = (inverse(gLead,p)*lead)%p
        a = {(degF-degG):gCoeff}
        newPoly = dictMult(g,a,p)
        q = dictAdd(q,a,p)
        newPoly = dictScal(newPoly,-1,p)
        f = dictAdd(f,newPoly,p)
        degF = degree(f)
        degG = degree(g)
    return f,q

In [22]:
quotient({3:1,2:8,1:12,0:4},{3:1,2:6,1:2,0:10},17)

({2: 2, 1: 10, 0: 11}, {0: 1})

In [23]:
def euclid(f,g,p):
    degF = degree(f)
    degG = degree(g)
    while degG>=0:
        r,q = quotient(f,g,p)
        print(f,"=",q,"*",g,"+",r)
        f,g=g,r
        degF = degree(f)
        degG = degree(g)
    return f


In [24]:
euclid({3:1,2:8,1:12,0:4},{3:1,2:6,1:2,0:10},109)

{3: 1, 2: 8, 1: 12, 0: 4} = {0: 1} * {3: 1, 2: 6, 1: 2, 0: 10} + {2: 2, 1: 10, 0: 103}
{3: 1, 2: 6, 1: 2, 0: 10} = {1: 55, 0: 55} * {2: 2, 1: 10, 0: 103} + {0: 13}
{2: 2, 1: 10, 0: 103} = {2: 84, 1: 93, 0: 75} * {0: 13} + {}


{0: 13}

In [25]:
euclid({3:1,2:2,1:6,0:8},{3:1,2:11,1:1,0:2},131)

{3: 1, 2: 2, 1: 6, 0: 8} = {0: 1} * {3: 1, 2: 11, 1: 1, 0: 2} + {2: 122, 1: 5, 0: 6}
{3: 1, 2: 11, 1: 1, 0: 2} = {1: 29, 0: 44} * {2: 122, 1: 5, 0: 6} + {}


{2: 122, 1: 5, 0: 6}

In [26]:
euclid({3:1,2:3,1:7,0:1},{3:1,2:3,1:4,0:12},157)

{3: 1, 2: 3, 1: 7, 0: 1} = {0: 1} * {3: 1, 2: 3, 1: 4, 0: 12} + {1: 3, 0: 146}
{3: 1, 2: 3, 1: 4, 0: 12} = {2: 105, 1: 72, 0: 56} * {1: 3, 0: 146} + {}


{1: 3, 0: 146}

In [27]:
# we want to find gcd(x^p-x,f) = gcd((x^p-x)%f,f)
def repeatSquareMod(p,f):
    n=p
    answer = {0:1}
    r={1:1}
    while n>0:
        r = quotient(r,f,p)[0]
        if r=={}:
            break
        if n%2!=0:
            n-=1
            answer = dictMult(answer,r,p)
        n//=2
        r = dictMult(r,r,p)
    answer = dictAdd(answer,{1:-1},p)
    answer = quotient(answer,f,p)[0]
    return answer

In [30]:
for i in range(1,5):
    print(repeatSquareMod(131,{2:1,0:-i}))

{}
{1: 129}
{}
{}
